In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv
/kaggle/input/en-core-web-md/en_core_web_md/meta.json
/kaggle/input/en-core-web-md/en_core_web_md/__init__.py
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/tokenizer
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/LICENSES_SOURCES
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/LICENSE
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/README.md
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/meta.json
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/accuracy.json
/kaggle/input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1/config.cfg
/kag

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
import spacy

nlp = spacy.load("../input/en-core-web-md/en_core_web_md/en_core_web_md-3.4.1")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
summary_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
prompt_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')

In [4]:
from string import punctuation as punc
from collections import defaultdict
import spacy
from string import punctuation as punc
from tqdm import tqdm
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import spacy
from textblob import TextBlob
import math
from collections import Counter

vector_size=256
window=10
train_df = summary_df.merge(right=prompt_df, how='inner', on='prompt_id')
sentences = train_df.apply(lambda x: x['prompt_title'].lower().split() + x['prompt_question'].lower().split() + x['prompt_text'].lower().split() + x['text'].lower().split(), axis=1)
model = Word2Vec(sentences, vector_size=vector_size, 
                 window=window, min_count=1, workers=4,sg=1)

def get_embeddings(text):
    word_vec = model.wv.get_mean_vector(text.lower().split())
    return  word_vec

def count_punc(text):
    return len([p for p in text if p in punc])

# Overlap Score
def overlap(sample):
    answer = set(sample['text'].lower().split())
    context = set(sample['prompt_text'].lower().split())
    intersection = len(context.intersection(answer))
    difference = len(context.difference(answer))
    union = len(context.union(answer))
    overlap_score = float(intersection/len(context))
    jaccard_similarity = float(intersection/union)
    return intersection, difference, union, overlap_score, jaccard_similarity

def tree_depth_from_token(token):
    """Calculate the depth of a subtree rooted at a token."""
    if not list(token.children):
        return 1
    else:
        return 1 + max(tree_depth_from_token(child) for child in token.children)

def average_dependency_tree_depth(doc):
    """Compute the average depth of the dependency tree."""
    root_tokens = [tok for tok in doc if tok.dep_ == 'ROOT']
    if not root_tokens:
        return 0
    depths = [tree_depth_from_token(token) for token in root_tokens]
    return sum(depths) / len(depths)

def extract_features(row):
    doc = nlp(row['text'].lower())
    context = nlp(row['prompt_text'].lower())
    
    # 1. Length of the summary
    length = len(doc)
    
    # 2. Number of unique words
    unique_words = len(set([token for token in doc]))
    context_unique_words = len(set([token for token in context]))
    
    #3. counter
    tokens = [token for token in doc]
    total_tokens = len(tokens)
    word_counts = Counter(tokens)
    once = sum(1 for word, count in word_counts.items() if count == 1)
    twice = sum(1 for word, count in word_counts.items() if count == 2)
    
    # 4. Named entities
    num_entities = len(doc.ents)
    context_num_entities = len(context.ents)
    
    # 5. Average word length
    avg_word_len = round(sum([len(token.text) for token in doc])/length, ndigits=4)
    context_word_len = round(sum([len(token.text) for token in context])/length, ndigits=4)
    
    # 6. Text Polarity
    sentiment_polarity = TextBlob(row['text']).sentiment.polarity
    
    # 7. part of speech tags
    num_nouns = len([token for token in doc if token.pos_ == "NOUN"])
    num_verbs = len([token for token in doc if token.pos_ == "VERB"])
    num_adverbs = len([token for token in doc if token.pos_ == "ADV"])
    
    # 8. Numerical entities
    num_numerical_entities = len([ent for ent in doc.ents if ent.label_ == "CARDINAL"])
    
    # 9. Number of sentence
    sentence = len([sent for sent in doc.sents])
    context_sentence = len([sent for sent in context.sents])
    avg_sentence = float(sentence / total_tokens)
    avg_context_sentence = float(context_sentence / context_unique_words)
    avg_unique_sentence = float(sentence / unique_words)
    
    # 10. root word depth
    average_tree_depth = average_dependency_tree_depth(doc)
    
    # 11. Probabilities of each word
    prob_word = [count/total_tokens for word, count in word_counts.items()]
    
    # 12. Stopwords
    stops = len([token for token in doc if token.is_stop])
    
    results = {
        "nsubj": 0,
        "amod": 0,
        "advmod": 0,
        "neg": 0,
        "xcomp": 0,
        "acomp": 0,
        'ROOT': 0
    }
    
    for token in doc:
        # Check if the token's dependency is one of the desired dependencies
        if token.dep_ in results:
            results[token.dep_] += 1
    
    # Organizing features in a dictionary
    features = {
        'length': length,
        'num_chars': len(row['text']),
        "stowords": stops,
        'sentence': sentence,
        "context_sentence": context_sentence,
        "avg_sentence": avg_sentence,
        "avg_context_sentence": avg_context_sentence,
        'avg_unique_sentence':avg_unique_sentence,
        'unique_words': unique_words,
        "context_unique_words": context_unique_words,
        'num_entities': num_entities,
        "context_num_entities": context_num_entities,
        'avg_word_len': avg_word_len,
        "context_word_len": context_word_len,
        "num_nouns": num_nouns,
        "num_verbs": num_verbs,
        "num_adverbs": num_adverbs,
        "num_numerical_entities": num_numerical_entities,
        "sentiment_polarity": sentiment_polarity,
        "nsubj": results['nsubj'],
        "amod": results['amod'],
        "advmod": results['advmod'],
        "neg": results['neg'],
        "xcomp": results['xcomp'],
        "acomp": results['acomp'],
        "average_dependency_tree_depth": average_tree_depth,
        "TTR": unique_words / (total_tokens + 1e-8),
        "RTTR": unique_words / (math.sqrt(total_tokens) + 1e-8),
        "CTTR": unique_words / (math.sqrt(total_tokens / 2) + 1e-8),
        # MATTR would require a window-based approach and is thus more involved
        "Guiraud's R": unique_words / (math.sqrt(total_tokens) + 1e-8),
        "Herdan's C": math.log(unique_words) / (math.log(total_tokens) + 1e-8),
        "Dugast's U": math.log(total_tokens)**2 / ((math.log(total_tokens) - math.log(unique_words)) + 1e-8),
        "Honoré's H": 100 * math.log(total_tokens) / (1 - once/(unique_words + 1e-8)),
        "Entropy": -sum(p * math.log(p) for p in prob_word), 
        "Sichel’s S": twice,
        "Simpson’s D": sum((count/total_tokens)**2 for count in word_counts.values())
    }
    
    return features


def compute_similarity(row):
    source_vector = model.wv.get_mean_vector(row['prompt_text'])
    answer_vector = model.wv.get_mean_vector(row['text'])
    embedding_similarity = cosine_similarity([source_vector], [answer_vector])[0][0]
    euclidean = euclidean_distances([source_vector], [answer_vector])[0][0]
    pearson = np.corrcoef(source_vector.ravel(), answer_vector.ravel())[0, 1]
    
    return pd.Series([embedding_similarity, euclidean, pearson])

In [5]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

def preprocess_data(summary_df, prompt_df):
    train_df = summary_df.merge(right=prompt_df, how='inner', on='prompt_id')

    print('processing text embeddings......')
    word_vec = np.asarray(train_df['text'].apply(get_embeddings).tolist(), dtype=np.float32)
    context_vec = np.asarray(train_df['prompt_text'].apply(get_embeddings).tolist(), dtype=np.float32)
    
    # processing lemmas
    print('extracting features.....')
    data = pd.DataFrame(train_df.apply(extract_features, axis=1).tolist())
    
    print('processing features.....')
    # Process other columns
    train_df['intersection'], train_df['difference'], train_df['union'], train_df['overlap_score'], train_df['jaccard_similarity'] = zip(*train_df.apply(overlap, axis=1).tolist())
    train_df['summary_punctuation'] = train_df['text'].apply(count_punc)

    cols = ['summary_punctuation','intersection','difference','union', 
            'overlap_score', 'jaccard_similarity',
            'content', 'wording']

    print('applying vectorizer.....')
    similarity_df = train_df.apply(compute_similarity, axis=1)
    similarity = similarity_df.values
    
    stacked_bag = np.hstack((context_vec,word_vec,similarity,data.values,train_df[cols].values))

    print('collecting outputs.........')
    student_ids = train_df['student_id'].values

    return stacked_bag, student_ids

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import time

start_time = time.time()

stacked_bag, student_ids = preprocess_data(summary_df,prompt_df)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Preprocessed function executed in: {elapsed_time:.2f} seconds")

processing text embeddings......
extracting features.....
processing features.....
applying vectorizer.....
collecting outputs.........
Preprocessed function executed in: 1502.75 seconds


In [7]:
X_train,X_test,y_train,y_test = train_test_split(stacked_bag[:,:-2] ,stacked_bag[:,-2:],test_size=0.1,shuffle=True,random_state=11)

In [8]:
print('done')

done


In [9]:
def mcrmse(y_true, y_pred):
    return np.mean(np.sqrt(np.mean((y_true - y_pred)**2, axis=0)))

mcrmse_scorer = make_scorer(mcrmse, greater_is_better=False)

In [10]:
from sklearn.model_selection import KFold
scores = []
best_params = {'max_depth': 11,'max_features': 0.6,'min_samples_split': 10,'n_estimators': 400}
model_rf = MultiOutputRegressor(RandomForestRegressor(**best_params,n_jobs=-1, random_state=48),n_jobs=-1)
score = []
kfold = KFold(n_splits=10, shuffle=True, random_state=64)
for train_idx, test_idx in tqdm(kfold.split(X_train, y_train), desc='training model'):
    X, y = X_train[train_idx], y_train[train_idx]
    test,test_y = X_train[test_idx], y_train[test_idx]
    model_rf.fit(X, y)
    score.append(mcrmse(test_y,model_rf.predict(test)))
scores.append(np.mean(score))
print('Mean Score: %.2f'%(np.mean(score)))

training model: 0it [00:00, ?it/s]/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_

Mean Score: 0.51


In [11]:
y_preds = model_rf.predict(X_test)

In [12]:
mcrmse(y_test,y_preds)

0.4975205479277523

In [13]:
X_train.shape

(6448, 557)

In [14]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

def preprocess_data(summary_df, prompt_df):
    train_df = summary_df.merge(right=prompt_df, how='inner', on='prompt_id')

    print('processing text embeddings......')
    word_vec = np.asarray(train_df['text'].apply(get_embeddings).tolist(), dtype=np.float32)
    context_vec = np.asarray(train_df['prompt_text'].apply(get_embeddings).tolist(), dtype=np.float32)
    
    # processing lemmas
    print('extracting features.....')
    data = pd.DataFrame(train_df.apply(extract_features, axis=1).tolist())
    
    print('processing features.....')
    # Process other columns
    train_df['intersection'], train_df['difference'], train_df['union'], train_df['overlap_score'], train_df['jaccard_similarity'] = zip(*train_df.apply(overlap, axis=1).tolist())
    train_df['summary_punctuation'] = train_df['text'].apply(count_punc)

    cols = ['summary_punctuation','intersection','difference','union', 
            'overlap_score', 'jaccard_similarity']

    print('applying vectorizer.....')
    similarity_df = train_df.apply(compute_similarity, axis=1)
    similarity = similarity_df.values
    
    stacked_bag = np.hstack((context_vec,word_vec,similarity,data.values,train_df[cols].values))

    print('collecting outputs.........')
    student_ids = train_df['student_id'].values

    return stacked_bag, student_ids

In [15]:
test_summary = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')
test_prompts = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')

In [16]:
test_df,ids = preprocess_data(test_summary, test_prompts)

processing text embeddings......
extracting features.....
processing features.....
applying vectorizer.....
collecting outputs.........


In [17]:
y_preds = model_rf.predict(test_df)

In [18]:
df_test = pd.DataFrame(y_preds,columns=['content','wording'],index=ids).reset_index()
df_test = df_test.rename({'index':'student_id'},axis=1)

In [19]:
def is_valid_float(x):
    return isinstance(x, float) and x == x  # This checks that x is not NaN since NaN != NaN in Python.

cols_to_check = ['wording', 'content']
df_test[cols_to_check] = df_test[cols_to_check].applymap(lambda x: x if is_valid_float(x) else 0.0)

df_test.to_csv("submission.csv", index=False)